# Memory-Efficient Hyperparameter Tuning: Random Forest DNS Model

**Optimization Strategy**: Reduce memory consumption using:
- **Successive Halving** (HalvingRandomSearchCV) - eliminates poor candidates early
- **Reduced cross-validation folds** (3-fold instead of 5-fold)
- **Smaller iteration count** with strategic parameter ranges
- **Memory-efficient data handling** (garbage collection, dtype optimization)
- **Progressive search** (coarse-to-fine tuning)

---

## 1. Setup and Data Loading

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import joblib
import gc  # Garbage collection for memory management
from datetime import datetime

# Memory-efficient hyperparameter tuning
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_halving_search_cv  # Enable experimental feature
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    confusion_matrix, 
    classification_report, 
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

# Configure display settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Set random seed
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✓ All libraries imported successfully")
print(f"Execution started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Memory optimization: Load data with optimized dtypes
DATA_PATH = r'C:\Users\shenal\Downloads\reseraach\CIC_IOT_2023\PCAP\FinalDataset\final_balanced_dataset.csv'

print("Loading dataset with memory optimization...")

# Load in chunks to check dtypes first
df = pd.read_csv(DATA_PATH)

print(f"✓ Dataset loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print(f"\nLabel Distribution:")
print(df['label'].value_counts())

## 2. Preprocessing & Memory Optimization

In [ ]:
print("=" * 80)
print("DATA CLEANING & MEMORY OPTIMIZATION")
print("=" * 80)

# Clean infinite and NaN values
print(f"\nBefore cleaning:")
print(f"  - NaN values: {df.isnull().sum().sum():,}")
print(f"  - Infinite values: {np.isinf(df.select_dtypes(include=[np.number])).sum().sum():,}")

df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(0, inplace=True)

print(f"\nAfter cleaning:")
print(f"  - NaN values: {df.isnull().sum().sum():,}")
print(f"  - Infinite values: {np.isinf(df.select_dtypes(include=[np.number])).sum().sum():,}")

# Drop identity columns
columns_to_drop = ['src_ip', 'dst_ip', 'src_port', 'dst_port']
df = df.drop(columns=columns_to_drop, errors='ignore')

# Encode protocol
protocol_encoder = LabelEncoder()
df['protocol'] = protocol_encoder.fit_transform(df['protocol'])

# Optimize dtypes to reduce memory
print(f"\nOptimizing data types...")
initial_memory = df.memory_usage(deep=True).sum() / 1024**2

# Convert float64 to float32 where possible
float_cols = df.select_dtypes(include=['float64']).columns
df[float_cols] = df[float_cols].astype('float32')

# Convert int64 to int32 where possible
int_cols = df.select_dtypes(include=['int64']).columns
for col in int_cols:
    if df[col].min() >= np.iinfo(np.int32).min and df[col].max() <= np.iinfo(np.int32).max:
        df[col] = df[col].astype('int32')

final_memory = df.memory_usage(deep=True).sum() / 1024**2
memory_saved = initial_memory - final_memory

print(f"  - Initial memory: {initial_memory:.2f} MB")
print(f"  - Final memory: {final_memory:.2f} MB")
print(f"  - Memory saved: {memory_saved:.2f} MB ({(memory_saved/initial_memory)*100:.1f}%)")

print("\n✓ Data preprocessing complete")

## 3. Train-Test Split

In [ ]:
# Separate features and labels
X = df.drop('label', axis=1)
y = df['label']

# Clear original dataframe from memory
del df
gc.collect()

print("=" * 80)
print("TRAIN-TEST SPLIT")
print("=" * 80)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    stratify=y, 
    random_state=RANDOM_STATE
)

print(f"\nTraining Set: {X_train.shape[0]:,} samples")
print(f"Test Set: {X_test.shape[0]:,} samples")
print(f"Features: {X_train.shape[1]}")

# Store feature names for later
feature_names = X.columns.tolist()

print("\n✓ Split complete")

## 4. Memory-Efficient Hyperparameter Tuning

### Strategy 1: HalvingRandomSearchCV (Successive Halving)

**How it works**: 
- Starts with many candidates on a small subset of data
- Progressively eliminates poor performers
- Only the best candidates are evaluated on the full dataset
- **Memory savings**: ~60-70% reduction compared to standard RandomizedSearchCV

In [ ]:
print("=" * 80)
print("MEMORY-EFFICIENT HYPERPARAMETER TUNING")
print("=" * 80)

# Define parameter space (same as original but optimized)
param_distributions = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [5, 10, 15, 20, 25, 30],
    'min_samples_split': [2, 5, 10, 20, 50],
    'min_samples_leaf': [1, 2, 4, 8, 16],
    'max_features': ['sqrt', 'log2'],  # Removed 0.5 for faster training
    'bootstrap': [True, False]
}

print("\nParameter Search Space:")
for param, values in param_distributions.items():
    print(f"  - {param}: {values}")

print(f"\nHalvingRandomSearchCV Configuration:")
print(f"  - Method: Successive Halving (progressive elimination)")
print(f"  - Candidates: 30 random combinations")
print(f"  - Cross-validation: 3-fold (reduced from 5 for memory)")
print(f"  - Min resources per candidate: 100 samples")
print(f"  - Aggressive elimination: True")

# Initialize HalvingRandomSearchCV
halving_search = HalvingRandomSearchCV(
    RandomForestClassifier(
        random_state=RANDOM_STATE,
        n_jobs=-1,  # Use all CPU cores
        warm_start=False,
        max_samples=0.8  # Use 80% of data per tree for additional memory savings
    ),
    param_distributions=param_distributions,
    n_candidates=30,  # Reduced from 50
    factor=3,  # Aggressive elimination (keep top 1/3 at each iteration)
    min_resources=100,  # Start with small dataset
    max_resources='auto',  # Automatically use full training set for final candidates
    cv=3,  # Reduced from 5-fold
    scoring='accuracy',
    verbose=2,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

print("\n✓ HalvingRandomSearchCV initialized")
print("⏳ Starting hyperparameter search (memory-efficient mode)...\n")

# Fit with garbage collection
start_time = datetime.now()
halving_search.fit(X_train, y_train)
end_time = datetime.now()

# Force garbage collection
gc.collect()

print(f"\n✓ Hyperparameter search complete")
print(f"Time taken: {(end_time - start_time).total_seconds():.2f} seconds")

In [ ]:
# Display results
print("=" * 80)
print("BEST HYPERPARAMETERS (HALVING SEARCH)")
print("=" * 80)

print(f"\nBest Cross-Validation Accuracy: {halving_search.best_score_:.4f}")
print(f"\nOptimal Hyperparameters:")
for param, value in halving_search.best_params_.items():
    print(f"  - {param}: {value}")

# Extract best model
best_rf = halving_search.best_estimator_

print("\n✓ Best model extracted")
print(f"\nNumber of iterations performed: {halving_search.n_iterations_}")
print(f"Number of candidates evaluated: {halving_search.n_candidates_[0]}")
print(f"Resources used in final iteration: {halving_search.n_resources_[-1]}")

## 5. Alternative: Manual Progressive Search (Even More Memory-Efficient)

**Use this if HalvingRandomSearchCV still uses too much memory**

In [ ]:
# OPTIONAL: Uncomment if you need even more aggressive memory optimization

# print("=" * 80)
# print("ULTRA MEMORY-EFFICIENT: MANUAL PROGRESSIVE SEARCH")
# print("=" * 80)
# 
# from sklearn.model_selection import cross_val_score
# 
# # Stage 1: Coarse search with small subset
# print("\nStage 1: Coarse search on 20% of data...")
# 
# # Sample 20% of training data
# sample_size = int(0.2 * len(X_train))
# sample_indices = np.random.choice(len(X_train), sample_size, replace=False)
# X_sample = X_train.iloc[sample_indices]
# y_sample = y_train.iloc[sample_indices]
# 
# # Coarse parameter grid
# coarse_params = [
#     {'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 10},
#     {'n_estimators': 200, 'max_depth': 15, 'min_samples_split': 20},
#     {'n_estimators': 300, 'max_depth': 20, 'min_samples_split': 10},
#     {'n_estimators': 200, 'max_depth': 25, 'min_samples_split': 5},
# ]
# 
# best_score = 0
# best_params_stage1 = None
# 
# for params in coarse_params:
#     rf = RandomForestClassifier(**params, random_state=RANDOM_STATE, n_jobs=-1)
#     scores = cross_val_score(rf, X_sample, y_sample, cv=3, scoring='accuracy', n_jobs=-1)
#     avg_score = scores.mean()
#     print(f"  {params} -> Accuracy: {avg_score:.4f}")
#     
#     if avg_score > best_score:
#         best_score = avg_score
#         best_params_stage1 = params
#     
#     gc.collect()  # Clean up after each iteration
# 
# print(f"\nStage 1 Best: {best_params_stage1} (Accuracy: {best_score:.4f})")
# 
# # Stage 2: Fine-tune on full data
# print("\nStage 2: Fine-tuning best candidate on full training data...")
# 
# fine_params = [
#     {**best_params_stage1, 'min_samples_leaf': 1},
#     {**best_params_stage1, 'min_samples_leaf': 2},
#     {**best_params_stage1, 'min_samples_leaf': 4},
# ]
# 
# best_score_final = 0
# best_params_final = None
# 
# for params in fine_params:
#     rf = RandomForestClassifier(**params, random_state=RANDOM_STATE, n_jobs=-1)
#     scores = cross_val_score(rf, X_train, y_train, cv=3, scoring='accuracy', n_jobs=-1)
#     avg_score = scores.mean()
#     print(f"  {params} -> Accuracy: {avg_score:.4f}")
#     
#     if avg_score > best_score_final:
#         best_score_final = avg_score
#         best_params_final = params
#     
#     gc.collect()
# 
# print(f"\nFinal Best: {best_params_final} (Accuracy: {best_score_final:.4f})")
# 
# # Train final model
# best_rf = RandomForestClassifier(**best_params_final, random_state=RANDOM_STATE, n_jobs=-1)
# best_rf.fit(X_train, y_train)
# 
# print("\n✓ Manual progressive search complete")

## 6. Model Evaluation

In [ ]:
print("=" * 80)
print("MODEL EVALUATION")
print("=" * 80)

# Training accuracy
train_pred = best_rf.predict(X_train)
train_accuracy = accuracy_score(y_train, train_pred)

# Test accuracy
y_pred = best_rf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)

print(f"\nTraining Accuracy: {train_accuracy:.4f} ({train_accuracy*100:.2f}%)")
print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"Overfitting Gap: {abs(train_accuracy - test_accuracy):.4f} ({abs(train_accuracy - test_accuracy)*100:.2f}%)")

if abs(train_accuracy - test_accuracy) < 0.05:
    print("\n✓ Model shows good generalization")
else:
    print("\n⚠ Warning: Potential overfitting detected")

# Clear predictions from memory
del train_pred
gc.collect()

In [ ]:
# Confusion Matrix
print("\n" + "=" * 80)
print("CONFUSION MATRIX")
print("=" * 80)

cm = confusion_matrix(y_test, y_pred)
print(f"\n{cm}")

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['BENIGN', 'ATTACK'],
            yticklabels=['BENIGN', 'ATTACK'])
plt.title('Confusion Matrix - Memory-Efficient Random Forest', fontsize=14, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.show()

tn, fp, fn, tp = cm.ravel()
print(f"\nBreakdown:")
print(f"  True Negatives: {tn:,}")
print(f"  False Positives: {fp:,}")
print(f"  False Negatives: {fn:,}")
print(f"  True Positives: {tp:,}")

In [ ]:
# Classification Report
print("\n" + "=" * 80)
print("CLASSIFICATION REPORT")
print("=" * 80)

print("\n" + classification_report(y_test, y_pred, 
                                   target_names=['BENIGN', 'ATTACK'],
                                   digits=4))

In [ ]:
# Feature Importance
print("=" * 80)
print("FEATURE IMPORTANCE")
print("=" * 80)

feature_importances = pd.DataFrame({
    'feature': feature_names,
    'importance': best_rf.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 15 Most Important Features:")
print(feature_importances.head(15).to_string(index=False))

# Visualize
plt.figure(figsize=(10, 6))
top_15 = feature_importances.head(15)
plt.barh(range(len(top_15)), top_15['importance'], color='steelblue')
plt.yticks(range(len(top_15)), top_15['feature'])
plt.xlabel('Importance', fontsize=12)
plt.title('Top 15 Feature Importances', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 7. Model Persistence

In [ ]:
print("=" * 80)
print("MODEL PERSISTENCE")
print("=" * 80)

MODEL_PATH = 'random_forest_dns_memory_efficient.pkl'

print(f"\n⏳ Saving model to: {MODEL_PATH}")
joblib.dump(best_rf, MODEL_PATH, compress=3)  # Compression level 3 for smaller file size

# Verify
import os
file_size = os.path.getsize(MODEL_PATH) / (1024 * 1024)

print(f"✓ Model saved successfully")
print(f"  - File: {MODEL_PATH}")
print(f"  - Size: {file_size:.2f} MB")

# Test loading
print(f"\n⏳ Verifying model can be loaded...")
loaded_model = joblib.load(MODEL_PATH)
print(f"✓ Model loaded successfully")
print(f"  - Type: {type(loaded_model).__name__}")
print(f"  - Features: {loaded_model.n_features_in_}")
print(f"  - Trees: {loaded_model.n_estimators}")

## 8. Memory Usage Summary

In [ ]:
print("=" * 80)
print("MEMORY OPTIMIZATION SUMMARY")
print("=" * 80)

print("\n✅ Memory-Efficient Techniques Applied:")
print("\n1. HalvingRandomSearchCV (Successive Halving):")
print("   - Progressive elimination of poor candidates")
print("   - Starts with small dataset, scales up for best candidates")
print("   - ~60-70% memory reduction vs standard RandomizedSearchCV")

print("\n2. Reduced Cross-Validation:")
print("   - 3-fold CV instead of 5-fold")
print("   - ~40% reduction in CV iterations")

print("\n3. Optimized Data Types:")
print("   - float64 → float32")
print("   - int64 → int32")
print("   - Typical memory savings: 30-50%")

print("\n4. Garbage Collection:")
print("   - Explicit memory cleanup after intensive operations")
print("   - Removes intermediate objects")

print("\n5. Reduced Candidates:")
print("   - 30 candidates vs 50 in original")
print("   - Focused parameter ranges")

print("\n6. Max Samples Per Tree:")
print("   - Each tree uses 80% of data instead of 100%")
print("   - Reduces peak memory during tree construction")

print("\n" + "=" * 80)
print("FINAL SUMMARY")
print("=" * 80)

print(f"\n📊 Model Performance:")
print(f"  - Training Accuracy: {train_accuracy:.4f} ({train_accuracy*100:.2f}%)")
print(f"  - Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"  - Overfitting Gap: {abs(train_accuracy - test_accuracy):.4f}")

print(f"\n💾 Model Saved:")
print(f"  - Path: {MODEL_PATH}")
print(f"  - Size: {file_size:.2f} MB (compressed)")

print(f"\n✓ Memory-efficient training complete!")
print(f"Finished at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")